In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams.update({
    'font.family': 'Arial',
    'font.size': 9,
    'axes.linewidth': 0.5,
    'pdf.fonttype': 42,
    'ps.fonttype': 42,
    'xtick.color': 'black',
    'ytick.color': 'black',
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'xtick.major.size': 3,
    'ytick.major.size': 3,
    'text.color': 'black',
    'axes.labelcolor': 'black',
    'axes.edgecolor': 'black'
})

# Input directories
cfz_dirs = {
    "0.01 µM": "CFz/dir1",
    "100 µM":  "CFz/dir2",
    "1000 µM": "CFz/dir3"
}
ffz_dirs = {
    "0.01 µM": "FFz/dir1",
    "100 µM":  "FFz/dir2",
    "1000 µM": "FFz/dir3"
}
fz_dirs = {
    "2.3 μM": ["Fz/dir1"],
    "0.766 μM": ["Fz/dir2"],
    "0.23 μM": ["Fz/dir3"]
}

COLOR_MAP = {
    "CFz": {
        "0.01 µM": "#c6dfee",
        "100 µM": "#4f9dc6",
        "1000 µM": "#0072B2",
    },
    "FFz": {
        "0.01 µM": "#b3e6d3",
        "100 µM": "#4fbf95",
        "1000 µM": "#009E73",
    },
    "Fz": {
        "2.3 μM": "#E6550D",
        "0.766 μM": "#FD8D3C",
        "0.23 μM": "#FDBE85"
    }
}

# Concentration labels
CFz_FFz_conc_labels = ["0.01 µM", "100 µM", "1000 µM"]
Fz_conc_labels = ["2.3 μM","0.766 μM","0.23 μM"]

# Helper Functions
def normalize_roi_names(df):
    df.columns = df.columns.str.replace("Mean\\(", "", regex=True)
    df.columns = df.columns.str.replace("\\)", "", regex=True)
    df.columns = df.columns.str.replace(":", "", regex=False)
    df.columns = df.columns.str.strip()
    return df

def find_file(folder_list, keyword):
    for folder in folder_list:
        if not os.path.exists(folder):
            continue
        for fname in os.listdir(folder):
            if keyword in fname and fname.endswith(".csv"):
                return os.path.join(folder, fname)
    return None

def build_data(dir_map, luc_label):
    dff_data, lf_data = [], []
    for conc_label, folder_list in dir_map.items():
        if not isinstance(folder_list, list):
            folder_list = [folder_list]
        l0_file = find_file(folder_list, "L0")
        dff_file = find_file(folder_list, "deltaLoverL_bessel_filtered")
        lf_file = find_file(folder_list, "luminescence_bessel_filtered")
        fluo_file = find_file(folder_list, "mNeonGreen")
        if not all([dff_file, lf_file, fluo_file]):
            print(f"Skipping {conc_label} due to missing files.")
            continue
        try:
            dff_df = normalize_roi_names(pd.read_csv(dff_file, index_col=0))
            lf_df = pd.read_csv(lf_file, index_col=0)
            fluo_df = pd.read_csv(fluo_file, index_col=0).drop(columns=["Label"], errors="ignore")
            lf_df = normalize_roi_names(lf_df)
            fluo_df = normalize_roi_names(fluo_df)
            common = set(dff_df.columns) & set(lf_df.columns) & set(fluo_df.columns)
            for cell in common:
                max_dff = dff_df[cell].max()
                max_lum = lf_df[cell].max()
                max_fluo = fluo_df[cell].max()
                if pd.notna(max_dff):
                    dff_data.append({"Concentration": conc_label, "Max ΔL/L": max_dff, "Luc": luc_label})
                if pd.notna(max_fluo) and max_fluo != 0:
                    lf_data.append({"Concentration": conc_label, "Max L/F": max_lum / max_fluo, "Luc": luc_label})
        except Exception as e:
            print(f"⚠️ Error processing {conc_label}: {e}")
    return {
        "Max ΔL/L": pd.DataFrame(dff_data),
        "Max L/F": pd.DataFrame(lf_data)
    }

cfz_data = build_data(cfz_dirs, "CFz")
ffz_data = build_data(ffz_dirs, "FFz")
fz_data = build_data(fz_dirs, "Fz")

# Output Individual plots
def plot_luc(df, key, label, luc_order, filename):
    is_fz_only = all(luc == "Fz" for luc, _ in luc_order)
    fig_width = 2 if is_fz_only else 4
    fig, ax = plt.subplots(figsize=(fig_width, 2.5))

    for i, (luc, conc) in enumerate(luc_order):
        sub_df = df[(df['Luc'] == luc) & (df['Concentration'] == conc)]
        x_vals = np.random.normal(i, 0.08, size=len(sub_df))
        ax.scatter(x_vals, sub_df[key], s=30, color=COLOR_MAP[luc][conc], edgecolor="black", linewidth=0.5, zorder=10)
        if len(sub_df[key]) > 0:
            mean = sub_df[key].mean()
            ci95 = 1.96 * sub_df[key].std(ddof=1) / np.sqrt(len(sub_df))
            print(f"{luc} {conc} {key}: {mean:.2f} ± {ci95:.2f} (n={len(sub_df)})")
            ax.errorbar(i, mean, yerr=ci95, fmt='none', ecolor='black', elinewidth=1.5, capsize=4, zorder=11)
            ax.hlines(mean, i - 0.2, i + 0.2, color='black', linewidth=1.5, zorder=12)

    ax.set_xticks(range(len(luc_order)))
    if is_fz_only:
        ax.set_xticklabels([conc for _, conc in luc_order], fontsize=10)
    else:
        ax.set_xticklabels([luc for luc, _ in luc_order], fontsize=10)
        group_centers = [(i + i + 1) / 2 for i in range(0, len(luc_order), 2)]
        group_labels = [luc_order[i][1] for i in range(0, len(luc_order), 2)]
        for x, label_text in zip(group_centers, group_labels):
            ax.text(x, -0.15, label_text, ha="center", va="top", fontsize=10, transform=ax.get_xaxis_transform())

    ax.set_ylabel(label, fontsize=11)
    ax.tick_params(axis='both', labelsize=10)
    sns.despine(ax=ax)
    plt.tight_layout()
    plt.savefig(filename + ".svg", transparent=True)
    plt.savefig(filename + ".png", dpi=600, transparent=True)
    plt.show()

cfzffz_order = [(luc, conc) for conc in CFz_FFz_conc_labels for luc in ["CFz", "FFz"]]
plot_luc(pd.concat([cfz_data["Max ΔL/L"], ffz_data["Max ΔL/L"]]), "Max ΔL/L", r"Max $\Delta L/L$", cfzffz_order, "CFz_FFz_Max_dL_over_L")
plot_luc(pd.concat([cfz_data["Max L/F"], ffz_data["Max L/F"]]), "Max L/F", r"L$_{max}$/F", cfzffz_order, "CFz_FFz_Lmax_over_F")

fz_order = [("Fz", conc) for conc in Fz_conc_labels]
plot_luc(fz_data["Max ΔL/L"], "Max ΔL/L", r"Max $\Delta L/L$", fz_order, "Fz_Max_dL_over_L")
plot_luc(fz_data["Max L/F"], "Max L/F", r"L$_{max}$/F", fz_order, "Fz_Lmax_over_F")